# **토큰화 코드**

In [8]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()

print("input_text_list:", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx , word in enumerate(input_text_list)}
idx2str = {idx:word for idx , word in enumerate(input_text_list)}

print("str2idx:", str2idx)
print("idx2str:", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids:", input_ids)

# 출력 결과
# input_text_list: ['나는', '최근', '파리', '여행을', '다녀왔다']
# str2idx: {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
# idx2str: {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
# input_ids: [0, 1, 2, 3, 4]

input_text_list: ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx: {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str: {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids: [0, 1, 2, 3, 4]


# **토큰 아이디에서 벡터로 변환**


In [9]:
import torch
import torch.nn as nn

embedding_dim = 16
# len(str2idx) = 전체 단어의 길이 추출
# str2idx: {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4} = 길이 5

# embedding_dim = 차원 = 16

# embed_layer = 위의 설정으로 임베딩 층을 만든다
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# torch.tensor = 텐서로 변환 / 리스트나 배열은 GPU 사용이 불가능해서 모델이 이해할 수 있게 바꿈
# torch.tensor(input_ids) = 토큰 아이디를 텐서로 변환
# embed_layer(torch.tensor(input_ids)) = 토큰 아이디를 16차원의 임의의 숫자 집합의 벡터로 변환
input_embeddings = embed_layer(torch.tensor(input_ids)) #(5, 16)

# unsqueeze = 텐서에 차원을 추가
# 딥러닝 모델은 데이터를 배치로 처리하기 때문에 한개가 있더라도 배치 형태로 만들어야 모델이 처리할 수 있다
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)
input_embeddings.shape

torch.Size([1, 5, 16])

# **절대적 위치 인코딩**

In [10]:
embedding_dim = 16
max_position = 12
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
# 기존의 임베딩 층에 추가할 위치 임베딩 층
# 위치 인코딩을 생성하는 위치 임베딩 층
# max_position = 12 = 최대 토큰 수 -> 12개
position_embed_layer = nn.Embedding(max_position, embedding_dim)

# torch.arange = 0 부터 시작하는 1차원 텐서 생성
# dtype=torch.long = long 타입으로 생성 -> 위치 인덱스를 나타내기에 알맞은 타입
# unsqueeze = 배치 처리하기 위해 차원 추가
position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
# 위치 아이디를 임베딩 층에 입력해 위치 인코딩 설정 (벡터)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids))
token_embeddings = token_embeddings.unsqueeze(0)

# 토큰 임베딩에 위치 인코딩을 더해 모델에 입력할 최종 입력 임베딩 준비
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

# **쿼리, 키, 값 벡터를 만드는 nn.Linear층**

In [11]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
# 학습 가능한 가중치를 만들기 위해 선형 층을 사용
# 모델이 학습을 통해 어떤 특징이 중요한지 알아서 학습하도록 도와줌
# 선형층은 단순히 가중치를 곱하는 연산이지만,
# 이 가중치가 학습됨에 따라 입력 벡터에서 중요한 특징을 강조하고, 덜 중요한 특징을 약화할 수 있다.

# embedding_dim = 입력차원, head_dim = 출력차원?
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
# 만들어낸 가중치에 임베딩을 통과시킴
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

# **스케일 점곱 방식의 어텐션**

In [12]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
  # querys.size(-1) querys의 마지막 차원의 크기를 가져옴
  # dim_k = 16과 같음(head_dim)
  dim_k = querys.size(-1) #16

  # @ 연산자는 행렬 곱셈(MatMul)을 수행하는 연산자
  # Python 3.5 이상에서는 NumPy와 PyTorch에서 @연산자가 matmul()과 동일하게 동작한다
  # transpose 두개의 차원을 서로 바꿔주는 PyTorch 함수 (안의 인자는 substring 작동방식과 유사)

  # 분산이 커지는 것을 방지하기 위해 임베딩 차원 수(dim_k)의 제곱근으로 나눈다 sqrt(dim_k)
  # 정규화 과정 진행
  scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
  # 소프트 맥스 함수를 사용하여 다중 분류의 여러 선형 방정식의 출력 값을 정규화 (확률 분포로 변환 -> 각 쿼리에 대해 모든 키의 중요도를 확률 값으로 변환)
  # dim=1 -> 시퀀스 길이 차원에 대해 정규화

  # 결과적으로 weights는 가중치 행렬이 된다
  weights = F.softmax(scores, dim=1)

  # 가중치와 값을 곱해 입력과 동일한 형태의 출력을 반환한다
  return weights @ values

# **어텐션 연산의 입력과 출력**

In [13]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


# **어텐션 연산을 수행하는 AttentionHead 클래스**

In [14]:
# 파이썬 상속
# PyTorch의 nn.Module 상속
class AttentionHead(nn.Module):
  # 생성자
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) #쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) #키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) #값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    # 위에서 관계도 계산을 위해 만들어둔 함수 재활용
    outputs = compute_attention(
        self.weight_q(querys), #쿼리 벡터
        self.weight_k(keys), #키 벡터
        self.weight_v(values), #값 벡터
        is_causal = self.is_causal
    )

    return outputs

# 클래스 생성
attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)
